<a href="https://colab.research.google.com/github/RaoSaeed836/LangChain/blob/main/Superior_University.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
!pip install langchain langchain-community openai huggingface_hub faiss-cpu pypdf unstructured sentence-transformers tiktoken chromadb


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 15.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 6.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 69.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 57.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.2/313.2 kB 30.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 77.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.6/19.6 MB 77.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 26.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 83.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.1/103.1 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [7]:
!pip install langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.6/70.6 kB 1.8 MB/s eta 0:00:00


After installing the library, you should be able to import `ChatOpenAI`.

In [75]:
from langchain_openai import ChatOpenAI
from langchain_community.vectorstores import FAISS,Chroma
from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader,TextLoader
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor


In [12]:
loader=PyPDFLoader("/content/Student-HandBook-RAK-Campus-1.pdf")
doc_load=loader.load()

In [13]:
print(doc_load[31].page_content)

30
UNIVERSITY ACADEMIC 
RULES AND REGULATIONS


In [15]:
spliter=RecursiveCharacterTextSplitter(
    chunk_size=400,
    chunk_overlap=0
)

In [17]:
split_text=spliter.split_documents(doc_load)

In [18]:
split_text[78].page_content

'• Non-returning of the temporarily issued material within due time\n• Misconduct with the SIRC staff\nLibrary Borrowing Rules\nAll registered members with valid Superior University Identification Card are entitled to borrow SIRC \nmaterials. The borrowing privileges for different membership are categorized as under:\nCategory                           Borrowing Privileges     Time Period'

In [19]:
embedding=OpenAIEmbeddings()

In [20]:
vector_store=FAISS.from_documents(split_text,embedding)

In [21]:
#Retriver
retrievers=vector_store.as_retriever(
     SearchType='mmr',
    search_kwargs={'k':2,'lambda_mult':0.5}
)

In [34]:
query="What is the attendance requirement for final exams?"

In [35]:
result=retrievers.invoke(query)

In [36]:
for i, doc in enumerate(result):
    print(f"\n--- Result {i+1} ---")
    print(doc.page_content)


--- Result 1 ---
attendance requirement and have cleared their semester dues. Under normal circumstances, there is no 
provision to retake the Final Term Examination. However, in case of exceptional circumstances, the case 
may be submitted to the Controller of Examinations along with certifiable documents, which after due 
consideration shall be forwarded to the concerned Committee for a decision.

--- Result 2 ---
Controller of Examination Office, well in advance. Students are required to take the Final Term Examination 
of each registered course, failing which a grade shall be awarded as per the earlier status of the course. 
Only those students shall be eligible to sit in the Final Term Examination who have met the minimum


In [26]:
vector_store.save_local("faiss_index")


In [28]:
# Create a FAISS vector store from the documentsa
embedding_model = OpenAIEmbeddings()
vectorstore = FAISS.from_documents(split_text, embedding_model)

In [29]:
base_retriever = vectorstore.as_retriever(search_kwargs={"k": 5})

In [30]:
# Set up the compressor using an LLM
llm = ChatOpenAI(model="gpt-3.5-turbo")
compressor = LLMChainExtractor.from_llm(llm)

In [31]:

# Create the contextual compression retriever
compression_retriever = ContextualCompressionRetriever(
    base_retriever=base_retriever,
    base_compressor=compressor
)

In [45]:
# Query the retriever
query = "what is the goal of superior university"
compressed_results = compression_retriever.invoke(query)

In [46]:
for i, doc in enumerate(compressed_results):
    print(f"\n--- Result {i+1} ---")
    print(doc.page_content)



--- Result 1 ---
Welcome to Superior University – where your journey to excellence begins.

--- Result 2 ---
Superior University will not be a factory of unemployed youth and we will nurture youth to become job creators instead of job seekers.

--- Result 3 ---
We believe disciplined students effectively manage organizations to deliver services and thrive in a prosperous manner. Students are, therefore,

--- Result 4 ---
Superior University has bagged a rank among the world’s top 401 to 600 universities


In [47]:
loader=TextLoader("/content/faiss_index/rag_course_catalog.txt")
text=loader.load()

In [50]:
print(type(text[0]))

<class 'langchain_core.documents.base.Document'>


In [71]:
spliter=RecursiveCharacterTextSplitter(
    chunk_size=185,
    chunk_overlap=0
)

In [72]:
split_text=spliter.split_documents(text)

In [73]:
print(split_text[1].page_content)

Course Code: CS102
Course Name: Programming Fundamentals
Credit Hours: 4
Description: Introduction to programming using Python; variables, loops, and functions.
---
Course Code: CS103


In [77]:
vector=Chroma.from_documents(split_text,embedding)

In [82]:
retriver=vector.as_retriever(
    SearchType='mmr',
    search_kwargs={'k':3,'lambda_mult':0.5}
)

In [84]:
query="what you have the programming fundamental subject"
result=retriver.invoke(query)

TypeError: Collection.query() got an unexpected keyword argument 'lambda_mult'

In [81]:
for i, doc in enumerate(result):
    print(f"\n--- Result {i+1} ---")
    print(doc.page_content)



--- Result 1 ---
Course Code: CS102
Course Name: Programming Fundamentals
Credit Hours: 4
Description: Introduction to programming using Python; variables, loops, and functions.
---
Course Code: CS103

--- Result 2 ---
Description: Android app development fundamentals using Java/Kotlin.
---
Course Code: CS201
Course Name: Artificial Intelligence
Credit Hours: 3

--- Result 3 ---
Description: Database design, SQL queries, and normalization techniques.
---
Course Code: CS106
Course Name: Operating Systems
Credit Hours: 3

--- Result 4 ---
Description: Lexical analysis, parsing, and code generation.
---
Course Code: CS303
Course Name: Internet of Things
Credit Hours: 3
